## Ideas para la ferretería con agentes de smolagent

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents smolagents[litellm] python-dotenv

In [ ]:
!python.exe -m pip install --upgrade pip

### Cargar la clave de hugging face desde el fichero .env

In [1]:
import pprint

In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Ejemplo de tool para la ferretería

Descartar LiteLLMModel porque falla a la hora de finalizar en el step correcto.

In [34]:
'''
from smolagents import LiteLLMModel, CodeAgent, tool

# Configurar el modelo
model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = LiteLLMModel(model_id='huggingface/'+model_id)
'''

Usar en su lugar HFApiModel que sí termina en el step adecuado.

In [78]:
from smolagents import HfApiModel, CodeAgent, tool

model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = HfApiModel(model_id=model_id)

In [90]:
# Definir tools
@tool
def pregunta(mensaje: str) -> str:
    """
    Preguntar al usuario para obtener datos adicionales.

    Args:
        mensaje: Pregunta para el usuario.
    """
    respuesta_usuario = input(f"{mensaje}\nRespuesta: ")
    return respuesta_usuario

@tool
def respuesta_fuera_del_tema(mensaje: str) -> str:
    """
    Respuesta cuando las preguntas no están relacionadas con asuntos de la ferretería.

    Args:
        mensaje: Respuesta a enviar
    """
    return mensaje

@tool
def final_answer(mensaje: str) -> str:
# def respuesta_final(mensaje: str) -> str:
    """
    Respuesta final a la pregunta inicial y finalización del bucle.

    Args:
        mensaje: Texto de la respuesta.
    """
    return mensaje



# Configurar el agente con la tool de Fibonacci que acabamos de definir
agent = CodeAgent(tools=[pregunta, respuesta_fuera_del_tema, final_answer], model=model, add_base_tools=True)
agent.max_steps = 8

In [91]:
tools = ', '.join(agent.tools.keys())
prompt4 = f'''\nEres un asistente experto en ferretería que responde de manera directa y eficiente solamente a temas de ferretería y a ningún otro tema.

## Sigue estas reglas:

1. Si necesitas información, usa la tool pregunta() sin explicar por qué.
2. Cuando tengas la información, usa la tool final_answer().
3. Cuando la respuesta contiene una secuencia de pasos, indícalo como una lista numerada.
4. NO ejecutes más código después de final_answer().
5. UTILIZA solo las siguientes tools definidas y NINGUNA otra más:
{tools}
6. Responde siempre en castellano.

## Ejemplo de flujo correcto:
Usuario: "Necesito colgar un cuadro"
pregunta("¿Cuál es el tamaño del cuadro?")
Usuario responde: "1.5m x 1m"
pregunta("¿Qué tipo de pared es donde vas a colgar el cuadro?")
Usuario responde: "ladrillo"
final_answer("Para colgar un cuadro de 1.5m x 1m en una pared de ladrillo, usa tornillos para ladrillo o anclajes de expansión. Mide y marca la pared con precisión y elige un soporte adecuado al peso.")

## Avisos
1. Cuando la pregunta no esté relacionada con temas de la ferretería, SIEMPRE DEBES usar la tool final_answer() y usa como parámetro el siguiente texto:
"Solo respondo a preguntas relacionadas con temas de la ferretería, ¿en qué puedo ayudarle?"
2. Si respondes a una pregunta no relacionada con temas de ferretería tendrás una multa de $10,000,000
3. No definas nuevas funciones python.
'''

'''
1. NO generes "thoughts" ni explicaciones antes de llamar a una función.

NO hagas esto:
Thought: Necesito saber el tamaño...
Code: pregunta()
'''

def set_system_prompt(agent):
    prompt_templates = agent.prompt_templates
    # agent.prompt_templates['system_prompt'] += prompt4
    agent.prompt_templates['system_prompt'] = prompt4

set_system_prompt(agent)

In [ ]:
# pprint.pprint(agent.prompt_templates['system_prompt'])

La siguiente llamada debería fallar porque no es un tema de ferretería

In [ ]:
result = agent.run(
    "Cuanto vale el término 10 de la serie de fibonacci",
)

print(result)

In [ ]:
result = agent.run(
    "Necesito colgar un cuadro",
)

print(result)

In [ ]:
result = agent.run(
    "¿Qué material necesito y cuánta cantidad para pintar puertas de mi casa?",
)

print(result)

In [ ]:
import pprint

pprint.pprint(agent.memory.steps)